Using lightgbm and simple date features, this script scores ~ 0.674 on Public LB.

### Load libraries and data

In [23]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [4]:
# check missing values per column
train.isnull().sum(axis=0)/train.shape[0]

ID             0.000000
datetime       0.000000
siteid         0.099896
offerid        0.000000
category       0.000000
merchant       0.000000
countrycode    0.000000
browserid      0.050118
devid          0.149969
click          0.000000
dtype: float64

### Clean Data and Create Features

In [5]:
train['siteid'].fillna(-999, inplace=True)
test['siteid'].fillna(-999, inplace=True)

train['browserid'].fillna("None", inplace=True)
test['browserid'].fillna("None", inplace=True)

train['devid'].fillna("None", inplace=True)
test['devid'].fillna("None", inplace=True)

In [6]:
# set datatime
train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

In [7]:
# create datetime variable
train['tweekday'] = train['datetime'].dt.weekday
train['thour'] = train['datetime'].dt.hour
train['tminute'] = train['datetime'].dt.minute

test['tweekday'] = test['datetime'].dt.weekday
test['thour'] = test['datetime'].dt.hour
test['tminute'] = test['datetime'].dt.minute

In [8]:
cols = ['siteid','offerid','category','merchant']

for x in cols:
    train[x] = train[x].astype('object')
    test[x] = test[x].astype('object')

In [12]:
cat_cols = cols + ['countrycode','browserid','devid']

In [15]:
for col in cat_cols:
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values) + list(test[col].values))
    train[col] = lbl.transform(list(train[col].values))
    test[col] = lbl.transform(list(test[col].values))

In [16]:
train.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click,tweekday,thour,tminute
0,IDsrk7SoW,2017-01-14 09:42:09,128865,784773,48,127,4,2,2,0,5,9,42
1,IDmMSxHur,2017-01-18 17:50:53,142053,157563,59,65,1,8,0,0,2,17,50
2,IDVLNN0Ut,2017-01-11 12:46:49,2618,458279,69,15,0,1,2,0,2,12,46
3,ID32T6wwQ,2017-01-17 10:18:43,243406,345067,117,507,2,2,1,0,1,10,18
4,IDqUShzMg,2017-01-14 16:02:33,154278,417948,36,276,3,8,0,0,5,16,2


### Model Training

In [19]:
cols_to_use = list(set(train.columns) - set(['ID','datetime','click']))

In [21]:
X_train, X_test, y_train, y_test = train_test_split(train[cols_to_use], train['click'], test_size = 0.5)

In [24]:
dtrain = lgb.Dataset(X_train, y_train)
dval = lgb.Dataset(X_test, y_test)

In [38]:
params = {
    
    'num_leaves' : 256,
    'learning_rate':0.03,
    'metric':'auc',
    'objective':'binary',
    'early_stopping_round': 40,
    'max_depth':10,
    'bagging_fraction':0.5,
    'feature_fraction':0.6,
    'bagging_seed':2017,
    'feature_fraction_seed':2017,
    'verbose' : 1
    
    
}

In [40]:
clf = lgb.train(params, dtrain,num_boost_round=500,valid_sets=dval,verbose_eval=20)

[20]	valid_0's auc: 0.97053
[40]	valid_0's auc: 0.969921
[60]	valid_0's auc: 0.970815
[80]	valid_0's auc: 0.971025
[100]	valid_0's auc: 0.971814
[120]	valid_0's auc: 0.972153
[140]	valid_0's auc: 0.972404
[160]	valid_0's auc: 0.972621
[180]	valid_0's auc: 0.97285
[200]	valid_0's auc: 0.972982
[220]	valid_0's auc: 0.973189
[240]	valid_0's auc: 0.973452
[260]	valid_0's auc: 0.973619
[280]	valid_0's auc: 0.97376
[300]	valid_0's auc: 0.973899
[320]	valid_0's auc: 0.974062
[340]	valid_0's auc: 0.974164
[360]	valid_0's auc: 0.97429
[380]	valid_0's auc: 0.974482
[400]	valid_0's auc: 0.974695
[420]	valid_0's auc: 0.974918
[440]	valid_0's auc: 0.975128
[460]	valid_0's auc: 0.975323
[480]	valid_0's auc: 0.975504
[500]	valid_0's auc: 0.975669


In [42]:
preds = clf.predict(test[cols_to_use])

In [44]:
sub = pd.DataFrame({'ID':test['ID'], 'click':preds})
sub.to_csv('lgb_pyst.csv', index=False) #0.67466